# MLP

refs: 
    
- MLP: https://panjeh.medium.com/scikit-learn-hyperparameter-optimization-for-mlpclassifier-4d670413042b

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer,KNNImputer

In [2]:
df_train = pd.read_csv("data/train.csv", index_col='PassengerId')
df_test = pd.read_csv("data/test.csv", index_col='PassengerId')
df_train.head(2)

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
PassengerId,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True


# Data Treatment

In [3]:
def feature_inputer(
    df:pd.DataFrame,
    to_mode = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP'],
    to_median = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
) -> pd.DataFrame:
    for col in to_mode:
        mode_inputer = SimpleImputer(strategy='most_frequent')
        df[col] = mode_inputer.fit_transform(df[[col]])

    
    for col in to_median:
        inputer = SimpleImputer(strategy='median')
        df[col] = inputer.fit_transform(df[[col]])
    return df

def dtype_memory_reducer(df: pd.DataFrame) -> pd.DataFrame:
    df['CryoSleep'] = df['CryoSleep'].astype(bool)
    df['VIP'] = df['VIP'].astype(bool)

    df['HomePlanet'] = df['HomePlanet'].astype('category')
    df['Destination'] = df['Destination'].astype('category')
    return df

def outliers_to_log(df:pd.DataFrame) -> pd.DataFrame:
    to_log = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    for col in to_log:
        df[col] = np.log(df[col] + 1)
    return df

def cabin_inputer(df:pd.DataFrame) -> pd.DataFrame:
    cabin_features = df['Cabin'].str.split("/",expand=True)[[0,2]].rename(columns={0:'Deck',2:'side'})
    df = pd.concat([df,cabin_features],axis=1)
    df['Deck'] = df['Deck'].fillna('G').replace("T","G")
    df = df.drop(['side','Cabin'],axis=1)
    return df

def vip_knn_input(df: pd.DataFrame) -> pd.DataFrame:
    inputer = KNNImputer(n_neighbors=5)
    df['VIP'] = inputer.fit_transform(df[['VIP','RoomService']])[:,0]
    return df

def treat_dataset(df:pd.DataFrame) -> pd.DataFrame:
    df = df.drop(['Name'],axis=1)
    df = cabin_inputer(df)
    df = feature_inputer(df)
    df = vip_knn_input(df)
    df['0_bills'] = df[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1) == 0
    df = outliers_to_log(df)
    df = dtype_memory_reducer(df)
    return df

def feature_enginnering(df: pd.DataFrame) -> pd.DataFrame:
    df = pd.get_dummies(df,columns=['HomePlanet','Destination','Deck'],drop_first=True)
    return df

In [4]:
df_train = treat_dataset(df_train.copy())
df_train = feature_enginnering(df_train)

In [5]:
X_train,y_train = df_train.drop('Transported',axis=1), df_train['Transported']

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold, GridSearchCV

## Tuning

In [17]:
params = {
    'hidden_layer_sizes': [(20,5,),(20,),(30,)],
    'activation': ['tanh', 'relu'],
    'solver': ['adam'],
    'alpha': [0.01, 0.05],
    'learning_rate': ['constant'],
}

In [18]:
grid = GridSearchCV(
    MLPClassifier(max_iter=1000, early_stopping=True),
    param_grid=params,
    scoring='accuracy',
    cv=KFold(5, shuffle=True, random_state=42),
    verbose=3,
    return_train_score=True
)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier wa

[CV 1/5] END activation=tanh, alpha=0.01, hidden_layer_sizes=(20, 5), learning_rate=constant, solver=adam;, score=(train=0.779, test=0.776) total time=   0.2s


c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier wa

[CV 2/5] END activation=tanh, alpha=0.01, hidden_layer_sizes=(20, 5), learning_rate=constant, solver=adam;, score=(train=0.793, test=0.772) total time=   0.6s


...

c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier wa

[CV 5/5] END activation=relu, alpha=0.05, hidden_layer_sizes=(30,), learning_rate=constant, solver=adam;, score=(train=0.777, test=0.782) total time=   0.2s


c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier wa

c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=MLPClassifier(early_stopping=True, max_iter=1000),
             param_grid={'activation': ['tanh', 'relu'], 'alpha': [0.01, 0.05],
                         'hidden_layer_sizes': [(20, 5), (20,), (30,)],
                         'learning_rate': ['constant'], 'solver': ['adam']},
             return_train_score=True, scoring='accuracy', verbose=3)

In [19]:
pd.DataFrame(grid.cv_results_)[['mean_train_score','std_train_score','mean_test_score','std_test_score']].loc[grid.best_index_]

mean_train_score    0.793857
std_train_score     0.008682
mean_test_score     0.787186
std_test_score      0.004636
Name: 8, dtype: float64

In [20]:
grid.best_params_

{'activation': 'relu',
 'alpha': 0.01,
 'hidden_layer_sizes': (30,),
 'learning_rate': 'constant',
 'solver': 'adam'}

In [21]:
model = grid.best_estimator_
model.fit(X_train, y_train)

c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier wa

c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


MLPClassifier(alpha=0.01, early_stopping=True, hidden_layer_sizes=(30,),
              max_iter=1000)

## Cross Validation

# Submission

In [23]:
X_test = treat_dataset(df_test)
X_test = feature_enginnering(X_test)
X_test = pd.DataFrame(scaler.fit_transform(X_test), index=X_test.index, columns=X_test.columns)
X_test.head(2)

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,0_bills,HomePlanet_Europa,HomePlanet_Mars,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G
PassengerId,,,,,,,,,,,,,,,,,,,
0013_01,1.330443,-0.114147,-0.132689,-0.64483,-0.646513,-0.628742,-0.661615,-0.624214,1.170830,-0.553131,-0.525314,-0.315862,0.634992,-0.30408,-0.300857,-0.244898,-0.341629,-0.714311,1.490179
0018_01,-0.751630,-0.684313,-0.132689,-0.64483,0.140685,-0.628742,2.226792,-0.624214,-0.854095,-0.553131,-0.525314,-0.315862,0.634992,-0.30408,-0.300857,-0.244898,-0.341629,1.399951,-0.671060


In [24]:
y_test = model.predict(X_test)

In [25]:
submission = pd.Series(y_test, index=[X_test.index]).to_frame('Transported')
submission.to_csv('data/submission_MLP_2022-11-05.csv')